# Практика
Используемые библиотеки

In [66]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [246]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [247]:
df = df.set_index("Показатель")
df.at["Цветность", "Результат анализа"] = 0
df

,Единица измерений,Результат анализа,Норматив
Показатель,,,
pH,единицы pH,8.4,в пределах 6-9
Запах,баллы,1,не более 2-3
Цветность,градусы,0,не более 30
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
Нитраты (по NO3),мг/дм3,24,не более 45
Фосфаты (P),мг/дм3,0.36,"не более 3,5"
Хлориды (Cl),мг/дм3,200,не более 350


In [248]:
df['Результат анализа'] = pd.to_numeric(df['Результат анализа'], downcast='float')

In [249]:
df['Норматив'] = df['Норматив'].replace(to_replace=r'(не более (\d-)?)', value='0<', regex=True)
df['Норматив'] = df['Норматив'].replace(to_replace=r'в пределах ', value='', regex=True)
df['Норматив'] = df['Норматив'].replace(to_replace=r',', value='.', regex=True)
df['Норматив'] = df['Норматив'].replace(to_replace=r'-', value='<', regex=True)
df_1 = df['Норматив'].str.split('<',expand=True)
df = pd.concat([df, df_1], axis=1)
df.rename(columns={0: 'МИН', 1: 'МАКС'}, inplace=True)
df

,Единица измерений,Результат анализа,Норматив,МИН,МАКС
Показатель,,,,,
pH,единицы pH,8.400,6<9,6,9
Запах,баллы,1.000,0<3,0,3
Цветность,градусы,0.000,0<30,0,30
Жёсткость,мг-эквл/дм3,9.200,7<10,7,10
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,0<1.5,0,1.5
Нитриты (по NO2),мг/дм3,0.017,0<3.3,0,3.3
Нитраты (по NO3),мг/дм3,24.000,0<45,0,45
Фосфаты (P),мг/дм3,0.360,0<3.5,0,3.5
Хлориды (Cl),мг/дм3,200.000,0<350,0,350


In [253]:
df.dtypes

Единица измерений     object
Результат анализа    float32
Норматив              object
МИН                   object
МАКС                  object
dtype: object

In [257]:
df[['МИН','МАКС']] = df[['МИН','МАКС']].astype('float')
df.dtypes

Единица измерений     object
Результат анализа    float32
Норматив              object
МИН                  float64
МАКС                 float64
dtype: object

In [259]:
df1 = df.query('`МИН` <= `Результат анализа` <= `МАКС`')
df2 = df.query('`Результат анализа` > `МАКС`')
df1['Итог'] = 'Допустимое значение'
df2['Итог'] = 'Отклонение'
all_results = pd.concat([df1,df2], axis=0)
all_results

C:\Users\Алексей\AppData\Local\Temp\ipykernel_4432\953739331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Итог'] = 'Допустимое значение'
C:\Users\Алексей\AppData\Local\Temp\ipykernel_4432\953739331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Итог'] = 'Отклонение'


,Единица измерений,Результат анализа,Норматив,МИН,МАКС,Итог
Показатель,,,,,,
pH,единицы pH,8.400,6<9,6.0,9.0,Допустимое значение
Запах,баллы,1.000,0<3,0.0,3.0,Допустимое значение
Цветность,градусы,0.000,0<30,0.0,30.0,Допустимое значение
Жёсткость,мг-эквл/дм3,9.200,7<10,7.0,10.0,Допустимое значение
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,0<1.5,0.0,1.5,Допустимое значение
Нитриты (по NO2),мг/дм3,0.017,0<3.3,0.0,3.3,Допустимое значение
Нитраты (по NO3),мг/дм3,24.000,0<45,0.0,45.0,Допустимое значение
Фосфаты (P),мг/дм3,0.360,0<3.5,0.0,3.5,Допустимое значение
Хлориды (Cl),мг/дм3,200.000,0<350,0.0,350.0,Допустимое значение


Вывод: По итогу у показателей "Нефть" и "Окисляемость перманганатная" имеется отклонение от норматива.	

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


Даже при 10 млн повторений вероятность получается 0.1714835, что эксперементально отличается от ответа 0.119

In [ ]:
import random


probability = ["апельсин"]*5 + ['яблоко']*4
n = 10000000

object =  pd.DataFrame({"obj1":list(map(lambda object: random.choice(probability), range(n))),
                  "obj2":list(map(lambda object: random.choice(probability), range(n))),
                  "obj3":list(map(lambda object: random.choice(probability), range(n))),
                 })
pr = len(object.query('obj1=="апельсин" & obj2=="апельсин" & obj3=="апельсин"'))/object.shape[0]
pr

0.1714835

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

При 10 млн повторений вероятность получается 0.233, что эксперементально подтверждается

In [ ]:
objects = []
n = 10000000

for i in range(n):
    pr = ["стандарт"]*7 + ['нестандарт']*3
    count = 1
    while pr:
        item = random.choice(pr)
        if item == "стандарт":
            pr.remove(item)
            count += 1
        else:
            objects.append(count)
            break
            
objects.count(2)/n

0.2334594